In [2]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
import faiss

# --------------------------------------------------
# 1) Carregar CSV de alimentos
# --------------------------------------------------
# Exemplo CSV:
# FOOD_NAME,FOOD_INGREDIENTS,FOOD_SERVING_SIZE,ENERGY (KCAL),PROTEIN (G),TOTAL LIPID (FAT) (G),CARBOHYDRATE,LANGUAGE
df = pd.read_parquet("../../datasets/fsda/fsda_insa_dataset_final.parquet")

# Para cada linha, vamos criar uma string que combine FOOD_NAME + FOOD_INGREDIENTS.
# (Assim o embedding captura melhor o contexto)
food_texts = (
    df["FOOD_NAME"].fillna("").astype(str) + " " +
    df["FOOD_INGREDIENTS"].fillna("").astype(str)
).tolist()

print(f"Total de alimentos no CSV: {len(food_texts)}")

# --------------------------------------------------
# 2) Criar modelo Sentence Transformers
# --------------------------------------------------
model_name = "sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1"
model = SentenceTransformer(model_name)

# --------------------------------------------------
# 3) Gerar embeddings para todos os alimentos
# --------------------------------------------------
# Saída: array NumPy (N x d), onde N é o # de alimentos e d é a dimensão do embedding
all_embeddings = model.encode(food_texts, convert_to_numpy=True, show_progress_bar=True)

# É recomendável normalizar os vetores se formos usar Faiss (cosine sim / IP)
# Faiss costuma trabalhar com "dot product" ou "L2 distance", então para simular a
# similaridade de cosseno, normalizamos cada vetor para norma=1
norms = np.linalg.norm(all_embeddings, axis=1, keepdims=True)
all_embeddings = all_embeddings / norms

print("Shape dos embeddings:", all_embeddings.shape)

# --------------------------------------------------
# 4) Construir índice Faiss
# --------------------------------------------------
d = all_embeddings.shape[1]  # dimensão dos embeddings
# Vamos usar IndexFlatIP (product = dot product = similar a cosseno se normalizado)
index = faiss.IndexFlatIP(d)

# Adicionar embeddings ao índice
index.add(all_embeddings.astype(np.float32))

print(f"Index size: {index.ntotal} embeddings")

# --------------------------------------------------
# 5) Função de busca
# --------------------------------------------------
def search_food(query_text, top_k=5):
    """
    Gera o embedding de 'query_text', faz busca no índice Faiss e
    retorna as top_k entradas mais semelhantes do CSV.
    """
    # 5.1) Embedding da consulta
    query_emb = model.encode(query_text, convert_to_numpy=True)

    # Normalizar
    norm_q = np.linalg.norm(query_emb)
    query_emb = (query_emb / norm_q).astype(np.float32).reshape(1, -1)  # shape (1, d)

    # 5.2) Consultar no índice
    # distances => shape (1, top_k)
    # indices => shape (1, top_k)
    distances, indices = index.search(query_emb, top_k)

    results = []
    for i in range(top_k):
        row_idx = indices[0][i]
        score = distances[0][i]  # dot product (similar a cos sim)

        # Recupera infos do df
        food_name = df["FOOD_NAME"].iloc[row_idx]
        ingredients = df["FOOD_INGREDIENTS"].iloc[row_idx]
        kcal = df["ENERGY (KCAL)"].iloc[row_idx]

        results.append({
            "rank": i+1,
            "score": float(score),
            "food_name": food_name,
            "ingredients": ingredients,
            "kcal": kcal
        })
    return results

# --------------------------------------------------
# 6) Exemplo de uso
# --------------------------------------------------
user_input = "Posso comer beef jerky se já consumi 1500 kcal hoje?"

# Abordagem simples: usar a frase toda.
# Se quiser, poderia extrair a substring “beef jerky” de outra forma (NER ou heurísticas).
results = search_food(user_input, top_k=5)

print("Resultados da busca:")
for r in results:
    print(f"{r['rank']}) score={r['score']:.3f}  name={r['food_name']}  kcal={r['kcal']}")
    # ou mostrar 'ingredients', etc.

# Exemplo de checagem do top-1
best = results[0]
if best["score"] < 0.3:
    print("Não encontrei nada relevante! Score muito baixo.")
else:
    print(f"Alimento top-1: {best['food_name']}, calorias: {best['kcal']}")


Total de alimentos no CSV: 457782


Batches: 100%|██████████| 14306/14306 [25:43<00:00,  9.27it/s]


Shape dos embeddings: (457782, 768)
Index size: 457782 embeddings
Resultados da busca:
1) score=0.451  name=Bife de vaca (valor médio de acém alcatra e lombo) frito com manteiga  kcal=201.0
2) score=0.443  name=Bife de vaca (valor médio de acém alcatra e lombo) frito sem molho  kcal=183.0
3) score=0.420  name=Bife de vaca (valor médio de acém alcatra e lombo) cru  kcal=122.0
4) score=0.414  name=Bife de vaca (valor médio de acém alcatra e lombo) grelhado  kcal=163.0
5) score=0.397  name=Bife de cavalo alcatra frita com manteiga  kcal=177.0
Alimento top-1: Bife de vaca (valor médio de acém alcatra e lombo) frito com manteiga, calorias: 201.0


In [5]:
np.save("food_embeddings.npy", all_embeddings)
faiss.write_index(index, "food_index.faiss")
print("Embeddings e índice FAISS guardados com sucesso.")

Embeddings e índice FAISS guardados com sucesso.
